In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

In [ ]:
# Simple corpus for example
corpus = ["I like going to the park", "I like playing football", "The park is near my house", 
          "Do you like going to the park?", "Playing at the park is fun", "Football is a popular sport"]

# Create a vocabulary dictionary
vocab = set()
for sentence in corpus:
    words = sentence.split()
    vocab.update(words)
    
print(vocab)

word2id = {word: i for i, word in enumerate(vocab)}
id2word = {i: word for word, i in word2id.items()}


In [ ]:
# Co-occurrence Matrix X
X = np.zeros((len(vocab), len(vocab)))

for sentence in corpus:
    words = sentence.split()
    for i, word in enumerate(words):
        for j in range(max(i - 1, 0), min(i + 2, len(words))):
            if i != j:
                X[word2id[word]][word2id[words[j]]] += 1

# Non-zero co-occurrences
non_zero_coocs = np.transpose(np.nonzero(X))

# Hyperparameters
vector_size = 20
iterations = 200
learning_rate = 0.05
x_max = 100
alpha = 0.75

# Weight function
def f(x):
    if x < x_max:
        return (x / x_max) ** alpha
    return 1

# Initialization
W = np.random.rand(len(vocab), vector_size) # center word vectors
W_tilde = np.random.rand(len(vocab), vector_size) # context word vectors
b = np.random.rand(len(vocab)) # bias for center words
b_tilde = np.random.rand(len(vocab)) # bias for context words

In [ ]:
# Training
for epoch in range(iterations):
    total_loss = 0
    for i, j in non_zero_coocs:
        weight = f(X[i, j])
        inner_cost = W[i, :].dot(W_tilde[j, :]) + b[i] + b_tilde[j] - np.log(X[i, j])
        cost = weight * (inner_cost ** 2)
        total_loss += 0.5 * cost

        # Gradients
        grad_cost = weight * inner_cost
        grad_wi = grad_cost * W_tilde[j, :] 
        grad_wj = grad_cost * W[i, :]
        grad_bi = grad_cost
        grad_bj = grad_cost

        # Update vectors and biases
        W[i, :] -= learning_rate * grad_wi
        W_tilde[j, :] -= learning_rate * grad_wj
        b[i] -= learning_rate * grad_bi
        b_tilde[j] -= learning_rate * grad_bj

    print(f"Epoch: {epoch+1}, Loss: {total_loss}")


In [ ]:
# Normalize vectors
W = normalize(W)
for i in range(len(vocab)):
    print(f"{id2word[i]}: {W[i, :]}")